## HW2 PageRank
106062314 蔡政諺

### Import libraries

In [1]:
from pyspark import SparkConf, SparkContext
import os

### Mapper1
將 input file 根據 ```"\n"``` 切成一行一行，然後把每行轉換成 ```(i, [j])``` 與 ```(i, [])``` 兩個 key-value pair。<br>
其中 i 是 source node ， j 是 destination node 。<br>
Ex. ```"0	1"``` → ```[(0, [1]), (0, [])]```

In [2]:
def mapper1(lines):
    lines = lines.split("\n")
    edges = []
    for line in lines:
        items = line.split("\t")
        edges.append((int(items[0]), [int(items[1])]))
        edges.append((int(items[1]), [])) # prevent nodes that have no out-degree disappear
    return edges

### Mapper2
根據 node 的總數，給每個 node 賦初始 r 值。<br>
Ex. ```x[0] = 2, n = 5``` → ```(2, 0.2)```

In [3]:
def mapper2(x, n):
    return (x[0], 1/n)

### Mapper3
對 source node 的 r 值進行分配。分為 2 個部分：<br>
(1) $\beta$<br>
如果 source node 有 out degree 的話，這部分會均分給所有的 destination node 。<br>
如果是dead end的話，這部分會均分給所有 node 。<br>
Ex1. ```(1, ([2, 3], 0.2))``` → ```[(2, 0.08), (3, 0.08)]```<br>
(2) $1-\beta$<br>
均分給所有的 node 。<br><br>
所有需要均分給所有 node 的部分都透過後面的 renormalize 處理。

In [4]:
def mapper3(x, n):
    beta = 0.8
    maplist = []
    maplist.append((x[0], 0)) # prevent nodes that have no in-degree disappear
    for node in x[1][0]:
        val = beta*x[1][1]/len(x[1][0])
        maplist.append((node, val))
    return maplist

### Mapper4
進行 renormalization 。 S 是目前算出的 r 值的總和，因此必須對每個 node 都加上 (1-S)/n ，總和才會為 1 。

In [5]:
def mapper4(x, S, n):
    return (x[0], x[1] + (1-S)/n)

### Reducer1
這個 reducer 有兩個用途：<br>
(1) 將同一個 source node 的 destination node(s) 合併成一個 list 。<br>
Ex1. ```(0, [1]), (0, [2])``` → ```(0, [1, 2])```<br>
Ex2. ```[(0, [1]), (0, []), (0, [2]), (0, []), (0, [3]), (0, [])]``` → ```(0, [1, 2, 3])```<br>
(2) 將分配給同一個 node 的 r 值加總。<br>
Ex. ```[(2, 0.08), (2, 0.032), (2, 0.008), (2, 0.008), (2, 0.008), (2, 0.008), (2, 0.008)]``` → ```(2, 0.152)```

In [6]:
def reducer1(x, y):
    return x + y

### Main function
(1) 將檔案 ```p2p-Gnutella04.txt``` 讀取，經過 ```mapper1``` 與 ```reducer1``` ，得到 ```edges``` ，也就是所有 source node 與 destination node 的相連關係。<br>
(2) 使用 ```count()``` 將 node 的總數存進 ```n``` 。<br>
(3) ```edges``` 經過 ```mapper2``` ，得到 ```r``` ，也就是所有 node 初始的 r 值。<br>
(4) 將 ```edges``` 與 ```r``` join 起來，經過 ```mapper3``` 與 ```reducer1``` 。<br>
(5) 使用 ```sum()``` 算出當前 r 值的總和，再經過 ```mapper4``` 做 renormalization ，即可得到總和為 1 的 r 值。(4) 與 (5) 共執行 20 次。<br>
(6) 使用 ```sortBy()``` 根據 r 值對所有 node 排序。

In [7]:
filename = "p2p-Gnutella04.txt"
sc.stop()
conf = SparkConf().setMaster("local").setAppName("pagerank")
sc = SparkContext(conf=conf)
edges = sc.textFile(filename).flatMap(mapper1)
edges = edges.reduceByKey(reducer1)
print("edges:", edges.take(10))
n = edges.count()
print("# of nodes:", n)
r = edges.map(lambda x: mapper2(x, n))
print("init:", r.take(10))

iterations = 20
for i in range(1, iterations+1):
    pairs = edges.join(r)
    pairs = pairs.flatMap(lambda x: mapper3(x, n))
    r = pairs.reduceByKey(reducer1)
    S = r.map(lambda x: x[1]).sum()
    r = r.map(lambda x: mapper4(x, S, n))
    print("iter", i, "/", iterations)
    # print(r.take(10))
    
r = r.sortBy(lambda x: -x[1])

print("Done.")

edges: [(0, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), (1, [2, 11, 12, 13, 14, 15, 16, 17, 18, 19]), (2, []), (3, [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), (4, []), (5, []), (6, []), (7, []), (8, [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]), (9, [])]
# of nodes: 10876
init: [(0, 9.194556822361162e-05), (1, 9.194556822361162e-05), (2, 9.194556822361162e-05), (3, 9.194556822361162e-05), (4, 9.194556822361162e-05), (5, 9.194556822361162e-05), (6, 9.194556822361162e-05), (7, 9.194556822361162e-05), (8, 9.194556822361162e-05), (9, 9.194556822361162e-05)]
iter 1 / 20
iter 2 / 20
iter 3 / 20
iter 4 / 20
iter 5 / 20
iter 6 / 20
iter 7 / 20
iter 8 / 20
iter 9 / 20
iter 10 / 20
iter 11 / 20
iter 12 / 20
iter 13 / 20
iter 14 / 20
iter 15 / 20
iter 16 / 20
iter 17 / 20
iter 18 / 20
iter 19 / 20
iter 20 / 20
Done.


### Output the result
將 r 值最大的 10 個 node 依序輸出。

In [12]:
out = r.take(10)
print(out)

[(1056, 0.0006321988095901941), (1054, 0.0006291557128603987), (1536, 0.0005239103397527083), (171, 0.0005116224706020384), (453, 0.0004956586476699697), (407, 0.0004848441996390269), (263, 0.000479619289318484), (4664, 0.0004704975514074376), (261, 0.00046289158656890185), (410, 0.0004615100382904272)]


In [14]:
f = open("./Outputfile.txt", 'w')
for pair in out:
    f.write("%d	%.60f\n" % (pair[0], pair[1]))
f.close()